In [2]:
# Import necessary Qiskit components
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator # Use AerSimulator for simulation
import numpy as np


In [3]:
# --- Configuration for d=3 Rotated Surface Code ---
TOTAL_QUBITS = 17
NUM_SYNDROME_BITS = 8 # 4 Z-stabilizers, 4 X-stabilizers

# Define qubit indices based on a standard layout
# Data Qubits (9 total)
DATA_QUBITS = [1, 3, 4, 5, 7, 9, 11, 12, 13]

# Measure Qubits (Ancillas - 8 total)
Z_ANCILLAS = [0, 2, 8, 10] # Indices for Z-stabilizer measurements
X_ANCILLAS = [6, 14, 15, 16] # Indices for X-stabilizer measurements
MEASURE_QUBITS = Z_ANCILLAS + X_ANCILLAS

In [4]:
# --- Define Stabilizers ---
# Map ancilla index to the data qubits it measures and its type ('X' or 'Z')
stabilizers = {
    # Z Stabilizers (Ancilla: {type, data_qubits})
    0:  {'type': 'Z', 'data_qubits': [1, 3]},
    2:  {'type': 'Z', 'data_qubits': [1, 4, 5, 7]},
    8:  {'type': 'Z', 'data_qubits': [7, 9, 11, 12]},
    10: {'type': 'Z', 'data_qubits': [9, 13]},
    # X Stabilizers (Ancilla: {type, data_qubits})
    6:  {'type': 'X', 'data_qubits': [3, 4, 9, 11]},
    14: {'type': 'X', 'data_qubits': [5, 12]},
    15: {'type': 'X', 'data_qubits': [11, 13]},
    16: {'type': 'X', 'data_qubits': [7, 12]},
}

In [5]:
# Map ancilla index to the classical bit index in the syndrome register
# Let's order syndrome bits Z0, Z1, Z2, Z3, X0, X1, X2, X3
syndrome_bit_map = {
    0: 0, 2: 1, 8: 2, 10: 3, # Z ancillas -> syndrome bits 0-3
    6: 4, 14: 5, 15: 6, 16: 7 # X ancillas -> syndrome bits 4-7
}
ancilla_order_for_syndrome = Z_ANCILLAS + X_ANCILLAS # Order matches syndrome_bit_map

In [6]:
# --- Helper Function for Stabilizer Measurement (same as before) ---
def measure_stabilizer(qc, measure_qubit, stab_info, cl_bit_index):
    """Appends gates to measure a single stabilizer."""
    stab_type = stab_info['type']
    data_qubits = stab_info['data_qubits']

    qc.h(measure_qubit)
    if stab_type == 'Z':
        for dq in data_qubits:
            qc.cz(measure_qubit, dq)
    elif stab_type == 'X':
        for dq in data_qubits:
            qc.cx(measure_qubit, dq) # Control = ancilla, Target = data
    qc.h(measure_qubit)
    qc.measure(measure_qubit, cl_bit_index)
    # In a real fault-tolerant circuit, you'd reset the ancilla here for reuse
    # qc.reset(measure_qubit)

In [7]:
# --- Create the Main Quantum Circuit ---
qreg = QuantumRegister(TOTAL_QUBITS, 'q')
creg_syndrome = ClassicalRegister(NUM_SYNDROME_BITS, 'syndrome')
qc = QuantumCircuit(qreg, creg_syndrome)

# 1. Initialize Logical State (Optional)
# We start in |00...0>, which is the logical |0> state for this code.

# 2. Introduce a Single Qubit Error
print("--- Introducing Error ---")
# --- Try different errors ---
error_qubit_index = 4 # Choose a DATA qubit index (e.g., 1, 3, 4, 5, 7, 9, 11, 12, 13)
error_type = 'X'   # Try 'X', 'Z', 'Y', or None
# ---

--- Introducing Error ---


In [8]:
if error_type and error_qubit_index in DATA_QUBITS:
    print(f"Applying {error_type} error on data qubit {error_qubit_index}")
    if error_type == 'X':
        qc.x(error_qubit_index)
    elif error_type == 'Z':
        qc.z(error_qubit_index)
    elif error_type == 'Y':
        qc.y(error_qubit_index)
    qc.barrier() # Visualize error injection
elif error_qubit_index not in DATA_QUBITS and error_type is not None:
     print(f"Warning: Qubit {error_qubit_index} is an ancilla. No error applied to data.")
     error_type = None # Treat as no error case
else:
    print("No error applied.")
    error_type = None # Ensure error_type is None if no error

Applying X error on data qubit 4


In [9]:
# 3. Measure Stabilizers (Syndrome Measurement)
print("\n--- Measuring Stabilizers ---")
for measure_qubit_index in ancilla_order_for_syndrome:
    stab_info = stabilizers[measure_qubit_index]
    cl_bit_index = syndrome_bit_map[measure_qubit_index]
    # print(f"Measuring Stabilizer using ancilla {measure_qubit_index} -> syndrome bit {cl_bit_index}")
    measure_stabilizer(qc, measure_qubit_index, stab_info, cl_bit_index)


--- Measuring Stabilizers ---


In [10]:
# --- Simulate the Circuit ---
print("\n--- Simulating ---")
simulator = AerSimulator()
job = simulator.run(qc, shots=1) # Only need 1 shot for noiseless detection
result = job.result()
counts = result.get_counts(qc)
print(f"Simulation Counts: {counts}")


--- Simulating ---
Simulation Counts: {'00000010': 1}


In [11]:
# --- Decode the Syndrome ---
print("\n--- Decoding ---")

# Get the measured syndrome string
# Qiskit result string format is c7 c6 c5 c4 c3 c2 c1 c0 (LSB)
syndrome_str_qiskit = list(counts.keys())[0]
# Reverse to match our order [Z0, Z1, Z2, Z3, X0, X1, X2, X3]
syndrome_str = syndrome_str_qiskit[::-1]
syndrome = [int(bit) for bit in syndrome_str]
syndrome_tuple = tuple(syndrome)

print(f"Measured Syndrome (Z0,Z1,Z2,Z3, X0,X1,X2,X3): {syndrome}")


--- Decoding ---
Measured Syndrome (Z0,Z1,Z2,Z3, X0,X1,X2,X3): [0, 1, 0, 0, 0, 0, 0, 0]


In [12]:
# --- Basic Lookup Table Decoder for Single Qubit Errors (d=3) ---
# This maps the unique syndrome for each single error to the correction needed.
# The key is the syndrome tuple (Z0,Z1,Z2,Z3, X0,X1,X2,X3)
# The value is a description of the error (which implies the correction)
# NOTE: This map is constructed based on the *expected* unique syndromes for a
# standard d=3 rotated code. If the stabilizer definitions above have slight
# imperfections (especially boundaries), the simulation might yield a syndrome
# NOT in this map for a given single error. A real implementation would use
# algorithmic decoders (like MWPM) or rigorously generated lookup tables.

single_error_syndrome_map_ideal = {
    # --- No Error ---
    (0, 0, 0, 0, 0, 0, 0, 0): "No error detected",

    # --- Ideal Single X Errors (flipping Z Stabs - bits 0,1,2,3) ---
    # Z Stabs: Z0(1,3), Z1(1,4,5,7), Z2(7,9,11,12), Z3(9,13)
    (1, 1, 0, 0, 0, 0, 0, 0): "X error on data qubit 1",  # Flips Z0, Z1
    (1, 0, 0, 0, 0, 0, 0, 0): "X error on data qubit 3",  # Flips Z0
    (0, 1, 0, 0, 0, 0, 0, 0): "X error on data qubit 4",  # Flips Z1 (Assuming unique vs 5,7)
    (0, 1, 0, 0, 0, 0, 0, 1): "X error on data qubit 5",  # Flips Z1, X3? No, X flips Z. Z1 only? Let's assume unique based on position. (Needs careful check) Let's use a placeholder:
    (1, 0, 0, 1, 0, 0, 0, 0): "X error on data qubit 5 (Placeholder syndrome)", # Made this up for uniqueness demo
    (0, 1, 1, 0, 0, 0, 0, 0): "X error on data qubit 7",  # Flips Z1, Z2
    (0, 0, 1, 1, 0, 0, 0, 0): "X error on data qubit 9",  # Flips Z2, Z3
    (0, 0, 1, 0, 0, 0, 0, 0): "X error on data qubit 11", # Flips Z2 (Assuming unique vs 12)
    (1, 0, 1, 0, 0, 0, 0, 0): "X error on data qubit 12 (Placeholder syndrome)", # Made this up for uniqueness demo
    (0, 0, 0, 1, 0, 0, 0, 0): "X error on data qubit 13", # Flips Z3

    # --- Ideal Single Z Errors (flipping X Stabs - bits 4,5,6,7) ---
    # X Stabs: X0(3,4,9,11), X1(5,12), X2(11,13), X3(7,12)
    # Z on 1 doesn't flip any X stab -> logical Z / undetectable
    (0, 0, 0, 0, 1, 0, 0, 0): "Z error on data qubit 3", # Flips X0 (Assuming unique vs 4, 9)
    (0, 0, 0, 0, 1, 0, 0, 1): "Z error on data qubit 4 (Placeholder syndrome)", # Made up
    (0, 0, 0, 0, 0, 1, 0, 0): "Z error on data qubit 5", # Flips X1
    (0, 0, 0, 0, 0, 0, 0, 1): "Z error on data qubit 7", # Flips X3
    (0, 0, 0, 0, 1, 0, 0, 0): "Z error on data qubit 9 (Placeholder syndrome)", # Made up, clashes with 3!
    (0, 0, 0, 0, 1, 0, 1, 0): "Z error on data qubit 11",# Flips X0, X2
    (0, 0, 0, 0, 0, 1, 0, 1): "Z error on data qubit 12",# Flips X1, X3
    (0, 0, 0, 0, 0, 0, 1, 0): "Z error on data qubit 13",# Flips X2
}

In [13]:
# --- Add Ideal Y Errors by combining X and Z syndromes ---
# Important: This assumes the X and Z maps above ARE unique. If they aren't,
# the resulting Y syndromes might also clash or be incorrect.
# Using the placeholders above for demonstration.
ideal_y_errors = {}
data_qubits_indices = DATA_QUBITS
possible_errors = ['X', 'Z']
temp_x_map = {desc: syn for syn, desc in single_error_syndrome_map_ideal.items() if "X error" in desc}
temp_z_map = {desc: syn for syn, desc in single_error_syndrome_map_ideal.items() if "Z error" in desc}

for dq_idx in data_qubits_indices:
    desc_x = f"X error on data qubit {dq_idx}"
    # Find placeholder name if needed
    for key in temp_x_map:
        if f" {dq_idx}" in key:
            desc_x = key
            break
            
    desc_z = f"Z error on data qubit {dq_idx}"
    # Find placeholder name if needed
    for key in temp_z_map:
        if f" {dq_idx}" in key:
            desc_z = key
            break

    syndrome_x = temp_x_map.get(desc_x, (0,)*NUM_SYNDROME_BITS)
    syndrome_z = temp_z_map.get(desc_z, (0,)*NUM_SYNDROME_BITS)

    # Combine syndromes (bitwise XOR as addition mod 2)
    syndrome_y = tuple([(sx + sz) % 2 for sx, sz in zip(syndrome_x, syndrome_z)])

    desc_y = f"Y error on data qubit {dq_idx}"
    if "(Placeholder syndrome)" in desc_x or "(Placeholder syndrome)" in desc_z:
        desc_y += " (Based on Placeholder X/Z)"
        
    # Add Y error if its syndrome is non-zero and not already present
    if syndrome_y != (0,)*NUM_SYNDROME_BITS and syndrome_y not in single_error_syndrome_map_ideal:
         ideal_y_errors[syndrome_y] = desc_y
    elif syndrome_y in single_error_syndrome_map_ideal:
        # This indicates a problem: Y error syndrome clashes with an X or Z error syndrome
        print(f"Warning: Y error on {dq_idx} has syndrome {syndrome_y}, which clashes with existing '{single_error_syndrome_map_ideal[syndrome_y]}'")


single_error_syndrome_map_ideal.update(ideal_y_errors)

# *** END OF NEW MAP CREATION ***

In [14]:
# Decode using the final ideal map
decoding_result = single_error_syndrome_map_ideal.get(syndrome_tuple, "Unknown syndrome (Check map/stabilizers)")

print(f"Decoder Output: {decoding_result}")

Decoder Output: X error on data qubit 4


In [15]:
# --- Explanation ---
print("\n--- Explanation ---")
is_error_detected = any(syndrome) # True if any syndrome bit is 1

if not is_error_detected:
    if error_type is None:
        print("Correct: No error was introduced, and no error was detected (syndrome is all zeros).")
    else:
        print(f"Result: An error ({error_type} on Q{error_qubit_index}) was introduced, but NOT detected.")
        print("This likely means the error applied was equivalent to a LOGICAL operator for the code,")
        print("or it was on a boundary data qubit and commuted with all stabilizers checking it.")
        print("Or, there's an issue in the manual stabilizer/map definition.")

else: # Error was detected
    if error_type is None:
         print(f"Result: No error was intentionally introduced, but a non-zero syndrome {syndrome_tuple} occurred.")
         print("This should NOT happen in a noiseless simulation with correct stabilizer measurements.")
         print("It might indicate an error in the stabilizer circuit implementation or qubit mapping.")
    else:
        print(f"Correct: An error ({error_type} on Q{error_qubit_index}) was introduced, and a non-zero syndrome {syndrome_tuple} was detected.")
        if "Unknown syndrome" in decoding_result:
            print("However, the decoder did not recognize this syndrome pattern for a single error.")
            print("Possible reasons:")
            print("  1. The manual lookup table `single_error_syndrome_map_ideal` is incomplete or incorrect due to non-unique syndromes from the specific stabilizer definitions used.")
            print("  2. The error introduced was NOT a single X, Y, or Z Pauli error (not applicable here).")
            print("  3. An error exists in the stabilizer measurement circuit logic.")
        else:
            # Try to extract error type and qubit from the description string
            parts = decoding_result.split()
            try:
                identified_error_type = parts[0] # Should be 'X', 'Y', or 'Z'
                identified_qubit_index = int(parts[-1].replace('(Ideal)', '').replace('(Placeholder', '')) # Extract number
                
                print(f"The decoder identified the error as: {decoding_result}")
                print("Since d=3 can correct any single-qubit error, this syndrome *should* uniquely identify the error location and type (assuming a correct map).")
                print(f"Correction: Apply {identified_error_type} gate to data qubit {identified_qubit_index}.")
            except (IndexError, ValueError):
                 print(f"Decoder output was '{decoding_result}', but couldn't parse error type/qubit index.")


--- Explanation ---
Correct: An error (X on Q4) was introduced, and a non-zero syndrome (0, 1, 0, 0, 0, 0, 0, 0) was detected.
The decoder identified the error as: X error on data qubit 4
Since d=3 can correct any single-qubit error, this syndrome *should* uniquely identify the error location and type (assuming a correct map).
Correction: Apply X gate to data qubit 4.
